In [ ]:
!python3.10 /Users/falaputin/virtual_assistant/ru-punctuation-truecase/src/prepare_data.py --data_dir /Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/tatoeba_dataset --num_samples -1 --percent_dev 0.2

In [16]:
from typing import List, Tuple
import pandas as pd
import numpy as np

from transformers import AutoModel, AutoTokenizer

In [ ]:
df = pd.read_parquet("/Users/falaputin/Downloads/cleaned_top300k_merged_database.parquet.gzip", engine="pyarrow")
df

Подготовка данных для hugging face transformers

Пример из hugging face

In [18]:
from pathlib import Path
import re
import numpy as np
from sklearn.model_selection import train_test_split

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs


In [19]:
texts, tags = read_wnut('/Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/wnut17train.conll')
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [20]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [2]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [27]:
len(train_encodings.tokens(3))

90

In [21]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [22]:
for token, label in zip(train_encodings.tokens(), train_labels[0]):
    print(token, label)

[CLS] -100
R 12
##T -100
@ 12
B -100
##ie -100
##bers -100
##F -100
##air -100
##ies -100
: 12
the 12
day 12
Justin 8
B 3
##ie -100
##ber -100
was 12
born 12
was 12
a 12
rainy 12
day 12
, 12
but 12
it 12
was 12
no 12
rain 12
, 12
NO 12
! 12
the 12
heaven 12
was 12
crying 12
cause 12
he 12
lost 12
his 12
most 12
. 12
. -100
. -100
[SEP] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100
[PAD] -100


Подготовка собственного датасета

In [10]:
def file2array(path_to_file: str) -> List[List[str]]:
    result = []
    with open(path_to_file, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            result.append(line.split(' '))
    return result


def textlabel2arrays(path_to_text: str, path_to_labels: str) -> Tuple[List[List[str]], List[List[str]]]:
    texts = []
    labels = []
    with open(path_to_text, 'r') as f_text:
        with open(path_to_labels, 'r') as f_labels:
            for line_text, line_labels in zip(f_text.readlines(), f_labels.readlines()):

                line_text = line_text.strip()
                line_labels = line_labels.strip()
                
                texts.append(line_text.split(' '))
                labels.append(line_labels.split(' '))
    return texts, labels

In [28]:
path_to_train_text = "/Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/tatoeba_dataset/text_train.txt"
path_to_train_labels = "/Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/tatoeba_dataset/labels_train.txt"
path_to_val_text = "/Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/tatoeba_dataset/text_dev.txt"
path_to_val_labels = "/Users/falaputin/virtual_assistant/ru-punctuation-truecase/data/tatoeba_dataset/labels_dev.txt"

train_texts, train_tags = textlabel2arrays(path_to_train_text, path_to_train_labels)
val_texts, val_tags = textlabel2arrays(path_to_val_text, path_to_val_labels)

unique_tags = set(tag for doc in train_tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

print(len(train_texts) == len(train_tags))
print(len(val_texts) == len(val_tags))

True
True


In [29]:
print(len(train_texts), len(train_tags))
print(len(val_texts), len(val_tags))

806458 806458
201614 201614


In [30]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [36]:
len(train_encodings.tokens(4))

157